In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

In [6]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
#Читаем только, то что нужно
# Избавляемся от дупликатов id в phone
gatrain = pd.read_csv('gdrive/My Drive/TalkingData/gender_age_train.csv', index_col='device_id')
gatest = pd.read_csv('gdrive/My Drive/TalkingData/gender_age_test.csv', index_col = 'device_id')
phone = pd.read_csv('gdrive/My Drive/TalkingData/phone_brand_device_model.csv')
phone = phone.drop_duplicates('device_id', keep='first').set_index('device_id')
events = pd.read_csv('gdrive/My Drive/TalkingData/events.csv', parse_dates=['timestamp'], index_col='event_id')
appevents = pd.read_csv('gdrive/My Drive/TalkingData/app_events.csv', usecols=['event_id','app_id','is_active'], dtype={'is_active':bool})
applabels = pd.read_csv('gdrive/My Drive/TalkingData/app_labels.csv')

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
gatrain['trainrow'] = np.arange(gatrain.shape[0])
gatest['testrow'] = np.arange(gatest.shape[0])
brandencoder = LabelEncoder().fit(phone.phone_brand)
phone['brand'] = brandencoder.transform(phone['phone_brand'])
gatrain['brand'] = phone['brand']
gatest['brand'] = phone['brand']
Xtr_brand = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.brand)))
Xte_brand = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.brand)))
print('Признаки для брэндов (размеры): train shape {}, test shape {}'.format(Xtr_brand.shape, Xte_brand.shape))

Признаки для брэндов (размеры): train shape (74645, 131), test shape (112071, 131)


In [9]:
m = phone.phone_brand.str.cat(phone.device_model)
modelencoder = LabelEncoder().fit(m)
phone['model'] = modelencoder.transform(m)
gatrain['model'] = phone['model']
gatest['model'] = phone['model']
Xtr_model = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.model)))
Xte_model = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.model)))
print('Признаки по моделям (размеры): train shape {}, test shape {}'.format(Xtr_model.shape, Xte_model.shape))

Признаки по моделям (размеры): train shape (74645, 1667), test shape (112071, 1667)


In [10]:
appencoder = LabelEncoder().fit(appevents.app_id)
appevents['app'] = appencoder.transform(appevents.app_id)
napps = len(appencoder.classes_)
deviceapps = (appevents.merge(events[['device_id']], how='left',left_on='event_id',right_index=True)
                       .groupby(['device_id','app'])['app'].agg(['size'])
                       .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                       .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                       .reset_index())
deviceapps.head()

,device_id,app,size,trainrow,testrow
0,-9222956879900151005,548,18,21594.0,NaN
1,-9222956879900151005,1096,18,21594.0,NaN
2,-9222956879900151005,1248,26,21594.0,NaN
3,-9222956879900151005,1545,12,21594.0,NaN
4,-9222956879900151005,1664,18,21594.0,NaN


In [11]:
d = deviceapps.dropna(subset=['trainrow'])
Xtr_app = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.app)), 
                      shape=(gatrain.shape[0],napps))
d = deviceapps.dropna(subset=['testrow'])
Xte_app = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.app)), 
                      shape=(gatest.shape[0],napps))
print('Данные приложений (форма): train shape {}, test shape {}'.format(Xtr_app.shape, Xte_app.shape))

Данные приложений (форма): train shape (74645, 19237), test shape (112071, 19237)


In [0]:
# Связываем лэйбл с девайс ID через transform по столбцу label_id
applabels = applabels.loc[applabels.app_id.isin(appevents.app_id.unique())]
applabels['app'] = appencoder.transform(applabels.app_id)
labelencoder = LabelEncoder().fit(applabels.label_id)
applabels['label'] = labelencoder.transform(applabels.label_id)
nlabels = len(labelencoder.classes_)

In [13]:
devicelabels = (deviceapps[['device_id','app']]
                .merge(applabels[['app','label']])
                .groupby(['device_id','label'])['app'].agg(['size'])
                .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                .reset_index())
devicelabels.head()

,device_id,label,size,trainrow,testrow
0,-9222956879900151005,117,1,21594.0,NaN
1,-9222956879900151005,120,1,21594.0,NaN
2,-9222956879900151005,126,1,21594.0,NaN
3,-9222956879900151005,138,2,21594.0,NaN
4,-9222956879900151005,147,2,21594.0,NaN


In [14]:
d = devicelabels.dropna(subset=['trainrow'])
Xtr_label = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.label)), 
                      shape=(gatrain.shape[0],nlabels))
d = devicelabels.dropna(subset=['testrow'])
Xte_label = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.label)), 
                      shape=(gatest.shape[0],nlabels))
print('Данные лейблов: train shape {}, test shape {}'.format(Xtr_label.shape, Xte_label.shape))

Данные лейблов: train shape (74645, 492), test shape (112071, 492)


In [15]:
Xtrain = hstack((Xtr_brand, Xtr_model, Xtr_app, Xtr_label), format='csr')
Xtest =  hstack((Xte_brand, Xte_model, Xte_app, Xte_label), format='csr')
print('Все свойства: train shape {}, test shape {}'.format(Xtrain.shape, Xtest.shape))

Все свойства: train shape (74645, 21527), test shape (112071, 21527)


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import GaussianDropout, GaussianNoise
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping
#from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
def rstr(df): return df.dtypes, df.head(3) ,df.apply(lambda x: [x.unique()]), df.apply(lambda x: [len(x.unique())]),df.shape

def batch_generator(X, y, batch_size, shuffle):
    # Генератор пакетов - взято отсюда (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

In [18]:
targetencoder = LabelEncoder().fit(gatrain.group)
y = targetencoder.transform(gatrain.group)
nclasses = len(targetencoder.classes_)

X_train, X_val, y_train, y_val = train_test_split(
    Xtrain, y, train_size=.98, random_state=10)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
def baseline_model():
    # Описание модели - glorot_normalizer 
    # Он рисует образцы из усеченного нормального распределения с центром в 0 с 
    # помощью stddev sqrt (2 / (fanin fanout)), где fanin-количество входных 
    # единиц в Тензоре веса, а fanout-количество выходных единиц в Тензоре веса.
    # ReLU
    model = Sequential()

    model.add(Dense(150, input_dim=X_train.shape[1], kernel_initializer='glorot_normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    
    model.add(Dense(2, kernel_initializer='glorot_normal'))
    pre = PReLU()
    model.add(pre)

    model.add(Dense(50, kernel_initializer='glorot_normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    
    model.add(Dense(12, kernel_initializer='glorot_normal', activation='softmax'))
    # Компилим
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model,pre

model,pre=baseline_model()

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

#checkpoint = ModelCheckpoint("weights.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

fit= model.fit_generator(generator=batch_generator(X_train, y_train, 64, True),
                         epochs=20,
                         samples_per_epoch=X_train.shape[0],
                         validation_data=(X_val.todense(), y_val), verbose=2,
                         callbacks=[early_stopping])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., epochs=20, validation_data=(matrix([[..., verbose=2, callbacks=[<keras.ca..., steps_per_epoch=73152)`


Epoch 1/20


In [0]:
model.save('gdrive/My Drive/TalkingData/model.h5')